In [1]:
import json
from cassis import *

In [2]:
#data_sample = cassis.load('../data/20240716_text2tech_entitylinking_sample/138_cas.json')
with open('../data/20240716_text2tech_entitylinking_sample/138_cas.json', 'rb') as f:
   cas_data = load_cas_from_json(f)
cas_data

In [3]:
vars(cas_data)

{'_typesystem': <cassis.typesystem.TypeSystem at 0x7fe09051dfa0>,
 '_lenient': False,
 '_sofas': {'_InitialView': Sofa(sofaNum=1, xmiID=5, sofaID='_InitialView', _sofaString='"Samsung takes aim at blood pressure monitoring with the Galaxy Watch Active - Samsung has launched a new smartwatch that might just take the health and wellness capabilities of the common wearable into new territory. In addition to standard features like sleep and heart rate tracking, the company claims its new Galaxy Watch Active will be able to keep tabs on blood pressure, promising a more complete picture of health taken from the user\'s wrist. Unveiled amid the fanfare of its first folding phone, the Galaxy Watch Active is a sport-oriented wristwatch with a 40-mm-diameter (1.57-in) face bearing a 1.1-inch (28-mm) color display coated in Gorilla Glass 3. The 20-mm (0.8 in) swappable straps bring the total weight to 25 g (0.9 oz), while available colors are silver, black, rose gold and sea green. The Galaxy Wat

In [4]:
print(dir(cas_data))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_view', '_copy', '_current_view', '_find_all_fs', '_get_feature_structures', '_get_feature_structures_in_range', '_get_next_sofa_num', '_get_next_xmi_id', '_lenient', '_serialize', '_sofa_num_generator', '_sofas', '_typesystem', '_views', '_xmi_id_generator', 'add', 'add_all', 'add_annotation', 'add_annotations', 'create_view', 'document_language', 'get_covered_text', 'get_document_annotation', 'get_sofa', 'get_view', 'remove', 'remove_annotation', 'select', 'select_all', 'select_covered', 'select_covering', 'sofa_array', 'sofa_mime', 'sofa_string', 'sofa_uri', 'sofas', 'to_json', 'to_xmi', 'typecheck', 'typesystem', 'views']


In [5]:
# Print the CAS sofa text (source text)
print("Text:", cas_data.sofa_string)

# Print all annotations in the CAS
for annotation in cas_data.select_all():
    print(f"Type: {annotation.type.name}")
    print(f"Begin: {annotation.begin}, End: {annotation.end}")
    print(f"Features: {annotation.__dict__}")

Text: "Samsung takes aim at blood pressure monitoring with the Galaxy Watch Active - Samsung has launched a new smartwatch that might just take the health and wellness capabilities of the common wearable into new territory. In addition to standard features like sleep and heart rate tracking, the company claims its new Galaxy Watch Active will be able to keep tabs on blood pressure, promising a more complete picture of health taken from the user's wrist. Unveiled amid the fanfare of its first folding phone, the Galaxy Watch Active is a sport-oriented wristwatch with a 40-mm-diameter (1.57-in) face bearing a 1.1-inch (28-mm) color display coated in Gorilla Glass 3. The 20-mm (0.8 in) swappable straps bring the total weight to 25 g (0.9 oz), while available colors are silver, black, rose gold and sea green. The Galaxy Watch Active runs Samsung's Tizen-based Wearable OS 4.0 and is powered by a 230 mAh battery, which is less than half that of a large-sized Galaxy Watch. Also onboard is 768 

AttributeError: 'de_tudarmstadt_ukp_clarin_webanno_api_type_Feature' object has no attribute 'begin'

In [6]:
# Print the CAS sofa text (source text)
print("Text:", cas_data.sofa_string)

# Print all annotations in the CAS with checks for 'begin' and 'end' attributes
for annotation in cas_data.select_all():
    print(f"Type: {annotation.type.name}")
    
    # Check and print 'begin' and 'end' attributes if they exist
    begin = getattr(annotation, 'begin', None)
    end = getattr(annotation, 'end', None)
    if begin is not None and end is not None:
        print(f"  Begin: {begin}, End: {end}")
    
    # Access each feature of the annotation safely
    for feature in annotation.type.all_features:
        feature_name = feature.name
        feature_value = annotation.get(feature_name)
        print(f"  {feature_name}: {feature_value}")

Text: "Samsung takes aim at blood pressure monitoring with the Galaxy Watch Active - Samsung has launched a new smartwatch that might just take the health and wellness capabilities of the common wearable into new territory. In addition to standard features like sleep and heart rate tracking, the company claims its new Galaxy Watch Active will be able to keep tabs on blood pressure, promising a more complete picture of health taken from the user's wrist. Unveiled amid the fanfare of its first folding phone, the Galaxy Watch Active is a sport-oriented wristwatch with a 40-mm-diameter (1.57-in) face bearing a 1.1-inch (28-mm) color display coated in Gorilla Glass 3. The 20-mm (0.8 in) swappable straps bring the total weight to 25 g (0.9 oz), while available colors are silver, black, rose gold and sea green. The Galaxy Watch Active runs Samsung's Tizen-based Wearable OS 4.0 and is powered by a 230 mAh battery, which is less than half that of a large-sized Galaxy Watch. Also onboard is 768 

In [18]:
import cassis

def cas_to_nif_with_uri(cas_data, document_url="http://example.org/document-1"):
    # Retrieve the document text
    text = cas_data.sofa_string
    text_length = len(text)
    
    # Start the NIF formatted output with the document's full text context
    nif_output = []
    nif_output.append(f'<{document_url}#char=0,{text_length}>')
    nif_output.append('        a                     nif:RFC5147String , nif:String , nif:Context ;')
    nif_output.append(f'        nif:beginIndex        "0"^^xsd:nonNegativeInteger ;')
    nif_output.append(f'        nif:endIndex          "{text_length}"^^xsd:nonNegativeInteger ;')
    nif_output.append(f'        nif:isString          "{text}"@en .\n')

    # Extract entity annotations
    for annotation in cas_data.select_all():
        if hasattr(annotation, 'begin') and hasattr(annotation, 'end'):
            # print(annotation)
            entity_text = text[annotation.begin:annotation.end]
            begin = annotation.begin
            end = annotation.end
            
            # Fetch the annotation-level URI for the named entity
            uri = getattr(annotation, 'identifier', None)
            if uri is None:
                uri = f"http://my-annotator.org/unknown/{entity_text.replace(' ', '_')}"  # Fallback URI format

            # Create NIF triples for the entity with its URI
            nif_output.append(f'<{document_url}#char={begin},{end}>')
            nif_output.append('        a                     nif:RFC5147String , nif:String ;')
            nif_output.append(f'        nif:anchorOf          "{entity_text}"@en ;')
            nif_output.append(f'        nif:beginIndex        "{begin}"^^xsd:nonNegativeInteger ;')
            nif_output.append(f'        nif:endIndex          "{end}"^^xsd:nonNegativeInteger ;')
            nif_output.append(f'        nif:referenceContext  <{document_url}#char=0,{text_length}> ;')
            nif_output.append(f'        itsrdf:taIdentRef     <{uri}> .\n')

    return "\n".join(nif_output)

# Load CAS JSON file
with open('../data/20240716_text2tech_entitylinking_sample/138_cas.json', 'rb') as f:
    cas_data = cassis.load_cas_from_json(f)

# Convert to NIF format with URIs
nif_formatted_text = cas_to_nif_with_uri(cas_data)
print(nif_formatted_text)


<http://example.org/document-1#char=0,3695>
        a                     nif:RFC5147String , nif:String , nif:Context ;
        nif:beginIndex        "0"^^xsd:nonNegativeInteger ;
        nif:endIndex          "3695"^^xsd:nonNegativeInteger ;
        nif:isString          ""Samsung takes aim at blood pressure monitoring with the Galaxy Watch Active - Samsung has launched a new smartwatch that might just take the health and wellness capabilities of the common wearable into new territory. In addition to standard features like sleep and heart rate tracking, the company claims its new Galaxy Watch Active will be able to keep tabs on blood pressure, promising a more complete picture of health taken from the user's wrist. Unveiled amid the fanfare of its first folding phone, the Galaxy Watch Active is a sport-oriented wristwatch with a 40-mm-diameter (1.57-in) face bearing a 1.1-inch (28-mm) color display coated in Gorilla Glass 3. The 20-mm (0.8 in) swappable straps bring the total weight 